In [ ]:
import numpy as np
from jesse import helpers, research

_, trading_1m = research.get_candles(
    "Binance Spot",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-04-20"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

trading_1m.shape


In [8]:
np.save("data/btc_1m.npy", trading_1m)

In [1]:
import numpy as np

candles = np.load("data/btc_1m.npy")
candles.shape

(2787840, 6)

In [2]:
from custom_indicators.toolbox.bar.merge import np_merge_bars

N = int(candles.shape[0] / 240)

new_candles = np_merge_bars(candles, N, lag=1, sign_first=True)
print(new_candles.shape)
np.save("data/btc_1m_m4h_lag_1_sign.npy", new_candles)
new_candles = np_merge_bars(candles, N, lag=5, sign_first=True)
print(new_candles.shape)
np.save("data/btc_1m_m4h_lag_10_sign.npy", new_candles)
new_candles = np_merge_bars(candles, N, lag=10, sign_first=True)
print(new_candles.shape)
np.save("data/btc_1m_m4h_lag_20_sign.npy", new_candles)

new_candles = np_merge_bars(candles, N, lag=1)
print(new_candles.shape)
np.save("data/btc_1m_m4h_lag_1.npy", new_candles)
new_candles = np_merge_bars(candles, N, lag=5)
print(new_candles.shape)
np.save("data/btc_1m_m4h_lag_10.npy", new_candles)
new_candles = np_merge_bars(candles, N, lag=10)
print(new_candles.shape)
np.save("data/btc_1m_m4h_lag_20.npy", new_candles)


  0%|          | 0/2776223 [00:00<?, ?it/s]

(11616, 6)


  0%|          | 0/2776219 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns


def plot_kde(array: np.ndarray):
    ret = np.log(array[1:]) - np.log(array[:-1])
    standard = (ret - ret.mean()) / ret.std()
    kurtosis = np.mean((standard - standard.mean()) ** 4)
    plt.figure(figsize=(16, 12))
    sns.kdeplot(standard, label="bar", color="blue")
    sns.kdeplot(
        np.random.normal(size=1000000), label="Normal", color="black", linestyle="--"
    )
    plt.xticks(range(-5, 6))
    plt.legend(loc=8, ncol=5)
    plt.title(
        f"bar_{array.shape[0]}_kurtosis_{kurtosis}",
        loc="center",
        fontsize=20,
        fontweight="bold",
        fontname="Times New Roman",
    )
    plt.xlim(-5, 5)
    plt.grid(1)
    plt.show()

In [ ]:
new_candles = np.load("data/btc_1m_merged_2h_lag_5.npy")
plot_kde(new_candles[:, 2])

In [ ]:
import numpy as np
import pandas as pd
from lightweight_charts import Chart

target = "5_sampEn_10"

df = pd.read_parquet("data/btc_1m_entropy.parquet").replace([np.inf, -np.inf], np.nan)
df[target] = df[target].fillna(df[target].max())
# df[target] = df["sampEn_20"] - df["sampEn_20"].shift(1)
df_draw = df.iloc[-10000:]

chart = Chart(inner_width=1, inner_height=0.8)
chart2 = chart.create_subchart(position="bottom", width=1, height=0.2, sync=True)

chart.set(df_draw[["date", "open", "high", "low", "close", "volume"]])

line = chart2.create_line(name=target)
line.set(df_draw[[target]])

chart.show(block=True)
